@author Gediyon M. Girma

TD(n) prediction: State value function prediction for a given policy

In [ ]:
import gym
import time
import itertools
import random
import numpy as np
env = gym.make('MountainCar-v0')

In [11]:
bins = (30, 30)

def discretize_observation(observation):
  """
  Discretizes the observation space.

  Args:
    observation: The observation to discretize.

  Returns:
    The discretized observation.
  """


  low = env.observation_space.low
  high = env.observation_space.high

  return tuple(np.digitize(observation[i], np.linspace(low[i], high[i], bins[i] + 1)) - 1 for i in range(2))


In [8]:
# TD(n) prediction algorithm

alpha = 0.1 # step size for incremental averaging
gamma = 1 #discount factor
n = 2 # number of steps

# formaulate the state space with every combination of the discritsized elements of the states
states = itertools.product(np.arange(bins[0]), np.arange(bins[1]))

policy = {}
V = {} # state value function



for state in states:
  V[state] = 0 # initialize the state-value function

  policy[state] = np.zeros((env.action_space.n)) # initialize the a random policy
  policy[state][np.random.randint(0, env.action_space.n)] = 1 



episodes = 5e4
start_timer = time.time()
episode = 1
while episode < episodes:
  episode_tracker = []

  # reset the environment
  obs = env.reset()
  state = discretize_observation(obs) # discretize the observation

  episode_tracker.append([0,state])
  T = float('inf') # terminal state

  t = 0

  while True:

    if t<T:
      action = np.random.choice(np.arange(env.action_space.n), p = policy[state]) # select an action

      episode_tracker[-1].append(action)

      obs, reward, done, info = env.step(action) # taking the action
      next_state = discretize_observation(obs) # next state

      episode_tracker.append([reward, next_state])

      if done:
        T = t+1

    tau = t - n + 1 # the time step whose state estimate is being updated (going n steps back)

    if tau >= 0:
      
      # compute the return
      G = sum([(gamma**(i-tau-1))*episode_tracker[i][0] for i in range(tau+1, min(tau+n,T)+1)]) 

      if tau + n < T:
        G += (gamma**n)*V[episode_tracker[tau+n][1]] # compute the discounted return

      # update the action-value function
      V[episode_tracker[tau][1]] += alpha * (G - V[episode_tracker[tau][1]]) 
      
    if tau == T-1:  # when the update time reaches terminal state, break
      break

    state = next_state  # update the state

    t += 1 # updating the time- step

  episode += 1
  if episode % 10000 == 0:
    end_timer = time.time()
    timer = end_timer - start_timer
    elapsed_time_struct = time.gmtime(timer)
    formatted_time = time.strftime("%H:%M:%S", elapsed_time_struct)
    print("Episode: ",episode, " time: ", formatted_time)

  if episode == episodes:
    print("done!")



Episode:  10000  time:  00:05:17
Episode:  20000  time:  00:10:10
Episode:  30000  time:  00:14:56
Episode:  40000  time:  00:19:53
Episode:  50000  time:  00:24:45
done!
